<a href="https://colab.research.google.com/github/Santhin/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install hyperopt

ERROR! Session/line number was not unique in database. History logging moved to new session 63


In [0]:
import pandas as pd
import numpy as np
import os
import datetime
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [8]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [9]:
ls

data/       day2.ipynb  day4.ipynb  logs/
day1.ipynb  day3.ipynb  LICENSE     README.md


In [0]:
train=pd.read_pickle('data/train.p')
test=pd.read_pickle('data/test.p')
X_train,y_train=train['features'],train['labels']
X_test, y_test=test['features'], test['labels']
if y_train.ndim==1: y_train=to_categorical(y_train)
if y_test.ndim==1: y_test=to_categorical(y_test)

input_shape=X_train.shape[1:]
num_classes= y_train.shape[1]

In [0]:
def train_model(model,X_train,y_train, params_fit={}):
       model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
       
       logdir=os.path.join('logs',datetime.datetime.now().strftime("%Y%m%d-$H%M%S"))
       tensorboard_callback=tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
       
       
       model.fit(
           X_train,
           y_train,
           batch_size=params_fit.get('batch_size',128),
           epochs=params_fit.get('epochs',5),
           verbose=params_fit.get('verbose',1),
           validation_data=params_fit.get('validation_data',(X_train,y_train)),
           callbacks=[tensorboard_callback]
                )
       return model

In [0]:
def predict(trained_model, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = trained_model.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:

def get_cnn_v5(input_shape,num_classes):
     return Sequential([
     Conv2D(filters=32, kernel_size=(3,3),activation='relu',input_shape=input_shape),
     Conv2D(filters=32, kernel_size=(3,3),activation='relu', padding='same'),
     MaxPool2D(),
     Dropout(0.3),


     Conv2D(filters=64, kernel_size=(3,3),activation='relu', padding='same'),
     Conv2D(filters=64, kernel_size=(3,3),activation='relu'),
     MaxPool2D(),
     Dropout(0.3),

     Conv2D(filters=64, kernel_size=(3,3),activation='relu', padding='same'),
     Conv2D(filters=64, kernel_size=(3,3),activation='relu'),
     MaxPool2D(),
     Dropout(0.3),
     
     Flatten(),

     Dense(1024, activation='relu'),            
     Dropout(0.3),

     Dense(1024, activation='relu'),            
     Dropout(0.3),

     Dense(num_classes, activation='softmax'),  
    ])




In [16]:
model=get_cnn_v5(input_shape,num_classes)
model_trained=train_model(model,X_train,y_train)
predict(model_trained,X_test,y_test)

Epoch 1/5
272/272 [==============================] - 6s 23ms/step - loss: 2.2931 - accuracy: 0.3456 - val_loss: 0.8930 - val_accuracy: 0.7297
Epoch 2/5
272/272 [==============================] - 6s 22ms/step - loss: 0.7269 - accuracy: 0.7634 - val_loss: 0.1899 - val_accuracy: 0.9396
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.3653 - accuracy: 0.8824 - val_loss: 0.1077 - val_accuracy: 0.9655
Epoch 4/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2521 - accuracy: 0.9224 - val_loss: 0.0534 - val_accuracy: 0.9846
Epoch 5/5
272/272 [==============================] - 6s 22ms/step - loss: 0.1897 - accuracy: 0.9431 - val_loss: 0.0377 - val_accuracy: 0.9883


0.9702947845804989

In [18]:
model_trained.evaluate(X_test,y_test)

138/138 [==============================] - 0s 4ms/step - loss: 0.1509 - accuracy: 0.9703


[0.15092234313488007, 0.9702947735786438]

In [0]:
def get_model(params):
     return Sequential([
     Conv2D(filters=32, kernel_size=(3,3),activation='relu',input_shape=input_shape),
     Conv2D(filters=32, kernel_size=(3,3),activation='relu', padding='same'),
     MaxPool2D(),
     Dropout(params['dropout_cnn_block_one']),


     Conv2D(filters=64, kernel_size=(3,3),activation='relu', padding='same'),
     Conv2D(filters=64, kernel_size=(3,3),activation='relu'),
     MaxPool2D(),
     Dropout(params['dropout_cnn_block_two']),

     Conv2D(filters=128, kernel_size=(3,3),activation='relu', padding='same'),
     Conv2D(filters=128, kernel_size=(3,3),activation='relu'),
     MaxPool2D(),
     Dropout(params['dropout_cnn_block_three']),
     
     Flatten(),

     Dense(1024, activation='relu'),            
     Dropout(params['dropout_dense_block_one']),

     Dense(1024, activation='relu'),            
     Dropout(params['dropout_dense_block_two']),

     Dense(num_classes, activation='softmax'),  
    ])


In [0]:
def func_obj(params):
       model=get_model(params)
       model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
       model.fit(
           X_train,
           y_train,
           batch_size=int(params.get('batch_size',128)),
           epochs=5,
           verbose=0,
                )
       
       score=model.evaluate(X_test,y_test,verbose=0)
       accuracy=score[1]
       print(params,'accurarcy={}'.format(accuracy))
       return {"loss":-accuracy, 'status':STATUS_OK, 'model':model}

In [35]:
space={
    'batch_size' : hp.quniform("batch_size",100,200,10),
    'dropout_cnn_block_one':hp.uniform("dropout_cnn_block_one",0.3,0.5),
    'dropout_cnn_block_two':hp.uniform("dropout_cnn_block_two",0.3,0.5),
    'dropout_cnn_block_three':hp.uniform("dropout_cnn_block_three",0.3,0.5),

    "dropout_dense_block_one":hp.uniform("dropout_dense_block_one",0.3,0.7),
    "dropout_dense_block_two":hp.uniform("dropout_dense_block_two",0.3,0.7),
}

best=fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 200.0, 'dropout_cnn_block_one': 0.35847474673292506, 'dropout_cnn_block_three': 0.45385352493222486, 'dropout_cnn_block_two': 0.4450899367604768, 'dropout_dense_block_one': 0.6769733494506012, 'dropout_dense_block_two': 0.3312183621873994}
accurarcy=0.8462585210800171
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.31411647163732354, 'dropout_cnn_block_three': 0.4154292571225029, 'dropout_cnn_block_two': 0.3721798097310315, 'dropout_dense_block_one': 0.30688554355952374, 'dropout_dense_block_two': 0.49033943697845195}
accurarcy=0.9625850319862366
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.3697775185470584, 'dropout_cnn_block_three': 0.3366705444863478, 'dropout_cnn_block_two': 0.30657612307171406, 'dropout_dense_block_one': 0.3940241042188785, 'dropout_dense_block_two': 0.3510221177141697}
accurarcy=0.9424036145210266
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.3760369845207626, 'dropout_cnn_block_three': 0.3166956432460852, 'dropout_cnn_block_two': 0.3643535